In [1]:
MONGO_URL="localhost:27017"
MONGO_INITDB_ROOT_USERNAME="root" # 
MONGO_INITDB_ROOT_PASSWORD="dr5[gnhn" # 
MONGO_BOT_USERNAME="bot"
MONGO_BOT_PASSWORD="botpsswd"

bot_credentials = dict(
    DATABASE_NAME = "bot_db",
    DATABASE_HOST = MONGO_URL,
    DATABASE_ADMIN_USERNAME = MONGO_BOT_USERNAME,
    DATABASE_ADMIN_PASSWORD = MONGO_BOT_PASSWORD
)

import sys,os
sys.path.append("..")


# database credentials extractor import
# from modules.db.dbconfig import root_credentials, bot_credentials
from modules.db.mongodb import MongoDriver

client = MongoDriver(bot_credentials).client
db_instance = MongoDriver(bot_credentials)
collection_name = "timer_data_collection"
user_id = 122267418
query = {"user_id": user_id}

print(client)
client.list_database_names()

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


['admin', 'config', 'local', 'test_db']

In [2]:
from modules.db.dbmodel import get_records

AttributeError: module 'time' has no attribute 'tzset'

In [3]:
get_timerdata_object(client, db_instance, collection_name, query, user_id).export()

NameError: name 'get_timerdata_object' is not defined

In [4]:
get_records(client, db_instance, collection_name, query, user_id)

NameError: name 'get_records' is not defined

In [5]:
conda list

# packages in environment at C:\Users\Teravolt\anaconda3\envs\vendorbot:
#
# Name                    Version                   Build  Channel
apscheduler               3.6.3                    pypi_0    pypi
asttokens                 2.0.5              pyhd8ed1ab_0    conda-forge
backcall                  0.2.0              pyh9f0ad1d_0    conda-forge
backports                 1.0                        py_2    conda-forge
backports.functools_lru_cache 1.6.1                      py_0    conda-forge
boost                     1.74.0           py38h1266d08_3    conda-forge
boost-cpp                 1.74.0               h54f0996_2    conda-forge
bzip2                     1.0.8                h8ffe710_4    conda-forge
ca-certificates           2021.10.8            h5b45459_0    conda-forge
cachetools                4.2.1                    pypi_0    pypi
cairo                     1.16.0            hba8bd2f_1007    conda-forge
certifi                   2021.10.8        py38haa244fe_1    cond

In [8]:
collection = client["timerbot_db"]["users_collection"]

data = {'_id': 122267418, 'user_id': 122267418, 'username': '@radical_subject', 'firstname': 'Oleg', 'lastname': 'Fedorov'}

In [12]:
print(collection.insert_one(data))

DuplicateKeyError: E11000 duplicate key error collection: timerbot_db.users_collection index: _id_ dup key: { _id: 122267418 }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 122267418}, 'errmsg': 'E11000 duplicate key error collection: timerbot_db.users_collection index: _id_ dup key: { _id: 122267418 }'}